In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'.
application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS','STATUS', 'INCOME_AMT'], inplace=True)

In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts[classification_counts > 1])

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() <= 1].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION','USE_CASE', 'ORGANIZATION'])

#Display the modified dataframe

print(application_df.head())

   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                       0                     1   
1   108590              1                       0                     0   
2     5000              0                       0                     0   
3     6692              1                       0                     0   
4   142590              1                       0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     0                    0                    0   
1                     0                    1                    0   
2                     0                    0                    0   
3                     0                    1                    0   
4                     0                    1                    0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                    0                    0                  

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Check the data types of X_train
print(X_train.dtypes)

ASK_AMT                      int64
APPLICATION_TYPE_Other       uint8
APPLICATION_TYPE_T10         uint8
APPLICATION_TYPE_T19         uint8
APPLICATION_TYPE_T3          uint8
                             ...  
USE_CASE_ProductDev          uint8
ORGANIZATION_Association     uint8
ORGANIZATION_Co-operative    uint8
ORGANIZATION_Corporation     uint8
ORGANIZATION_Trust           uint8
Length: 71, dtype: object


In [12]:
# Check the data types of X_test
print(X_test.dtypes)

ASK_AMT                      int64
APPLICATION_TYPE_Other       uint8
APPLICATION_TYPE_T10         uint8
APPLICATION_TYPE_T19         uint8
APPLICATION_TYPE_T3          uint8
                             ...  
USE_CASE_ProductDev          uint8
ORGANIZATION_Association     uint8
ORGANIZATION_Co-operative    uint8
ORGANIZATION_Corporation     uint8
ORGANIZATION_Trust           uint8
Length: 71, dtype: object


In [13]:
# Create a StandardScaler instances
X_scaler = StandardScaler()

# Fit the StandardScaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
print(X_train_scaled.shape)

(25724, 71)


Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
# Get the number of input features (columns) from the shape of X_train_scaled
input_features = X_train_scaled.shape[1]

# Define the number of nodes for each hidden layer
hidden_nodes_layer1 = 48
hidden_nodes_layer2 = 24

# Define the model - deep neural net
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                3456      
                                                                 
 dense_1 (Dense)             (None, 24)                1176      
                                                                 
 dense_2 (Dense)             (None, 1)                 25        
                                                                 
Total params: 4,657
Trainable params: 4,657
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 7ms/step - loss: 0.5800 - accuracy: 0.7170
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5619 - accuracy: 0.7252
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5588 - accuracy: 0.7250
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5578 - accuracy: 0.7254
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5559 - accuracy: 0.7261
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5554 - accuracy: 0.7261
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7276
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7282
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7279
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5590 - accuracy: 0.7300 - 502ms/epoch - 2ms/step
Loss: 0.5590354800224304, Accuracy: 0.7300291657447815


In [19]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")